# CVL model

[Customer value in non-contractual business model](https://towardsdatascience.com/whats-a-customer-worth-8daf183f8a4f)

[lifetimes python package](https://github.com/CamDavidsonPilon/lifetimes) for modeling customer lifetime value (CVL).

[CVL in pyspark](http://webdropin.com/wordpress99/customer-lifetime-value-in-pyspark/)

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from lifetimes import BetaGeoFitter

from lifetimes.plotting import plot_frequency_recency_matrix

import helper as h

In [2]:
#users = pd.read_csv('all_users.csv', index_col=0)

In [3]:
#users.head()

In [4]:
user_data = h.get_merged_data()
user_data.head()

,login,followers_count,following_count,blog,company,created_at,public_repos_count,public_gists_count,hireable,updated_at,...,GollumEvent_count,IssueCommentEvent_count,IssuesEvent_count,MemberEvent_count,PublicEvent_count,PullRequestEvent_count,PullRequestReviewCommentEvent_count,PushEvent_count,ReleaseEvent_count,WatchEvent_count
0,0loky0,0,1,0,0,2011-05-24 20:15:25+00:00,7.0,0.0,1,2019-01-09 15:03:59+00:00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0
1,13246864965,0,0,0,0,2016-04-14 10:04:50+00:00,3.0,0.0,0,2018-08-02 07:48:44+00:00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1xPdd,0,0,0,0,2014-07-11 03:45:56+00:00,0.0,0.0,0,2018-12-16 00:55:16+00:00,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2582056905xy,2,6,0,0,2014-04-24 14:51:26+00:00,35.0,0.0,0,2018-10-18 01:47:18+00:00,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,2.0
4,277542952,1,28,0,0,2016-03-16 03:17:58+00:00,1.0,0.0,0,2017-08-14 19:48:16+00:00,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0


In [28]:
#pd.to_datetime(users['created_at'], errors='coerce')
user_data['created_at'] = pd.to_datetime(user_data.created_at, errors='coerce')
end_period = pd.to_datetime('2016-06-01 23:59:59+00:00') 
print(end_period)
user_data['T'] = np.round((end_period - user_data.created_at) / pd.Timedelta(days = 1))
user_data.head()

2016-06-01 23:59:59


,login,followers_count,following_count,blog,company,created_at,public_repos_count,public_gists_count,hireable,updated_at,...,IssuesEvent_count,MemberEvent_count,PublicEvent_count,PullRequestEvent_count,PullRequestReviewCommentEvent_count,PushEvent_count,ReleaseEvent_count,WatchEvent_count,T,recency
0,0loky0,0,1,0,0,2011-05-24 20:15:25,7.0,0.0,1,2019-01-09 15:03:59+00:00,...,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,1835.0,1787.0
1,13246864965,0,0,0,0,2016-04-14 10:04:50,3.0,0.0,0,2018-08-02 07:48:44+00:00,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,49.0,0.0
2,1xPdd,0,0,0,0,2014-07-11 03:45:56,0.0,0.0,0,2018-12-16 00:55:16+00:00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,692.0,683.0
3,2582056905xy,2,6,0,0,2014-04-24 14:51:26,35.0,0.0,0,2018-10-18 01:47:18+00:00,...,0.0,0.0,0.0,1.0,0.0,2.0,0.0,2.0,769.0,733.0
4,277542952,1,28,0,0,2016-03-16 03:17:58,1.0,0.0,0,2017-08-14 19:48:16+00:00,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,78.0,69.0


In [29]:
# move this to helper function!!
user_data['recency'] = (pd.to_datetime(user_data.last_event, errors='coerce') - 
                        user_data.created_at)  / pd.Timedelta(days=1)

In [45]:
user_data.head()

,login,followers_count,following_count,blog,company,created_at,public_repos_count,public_gists_count,hireable,updated_at,...,IssuesEvent_count,MemberEvent_count,PublicEvent_count,PullRequestEvent_count,PullRequestReviewCommentEvent_count,PushEvent_count,ReleaseEvent_count,WatchEvent_count,T,recency
0,0loky0,0,1,0,0,2011-05-24 20:15:25,7.0,0.0,1,2019-01-09 15:03:59+00:00,...,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,1835.0,1787.0
1,13246864965,0,0,0,0,2016-04-14 10:04:50,3.0,0.0,0,2018-08-02 07:48:44+00:00,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,49.0,0.0
2,1xPdd,0,0,0,0,2014-07-11 03:45:56,0.0,0.0,0,2018-12-16 00:55:16+00:00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,692.0,683.0
3,2582056905xy,2,6,0,0,2014-04-24 14:51:26,35.0,0.0,0,2018-10-18 01:47:18+00:00,...,0.0,0.0,0.0,1.0,0.0,2.0,0.0,2.0,769.0,733.0
4,277542952,1,28,0,0,2016-03-16 03:17:58,1.0,0.0,0,2017-08-14 19:48:16+00:00,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,78.0,69.0


In [62]:
user_data['recency'] = np.floor(user_data.recency)
lifetime_df = user_data[['frequency', 'recency', 'T']]
lifetime_df = lifetime_df.dropna()

In [63]:
lifetime_df.head()

,frequency,recency,T
0,19.0,1787.0,1835.0
1,3.0,0.0,49.0
2,3.0,683.0,692.0
3,11.0,733.0,769.0
4,14.0,69.0,78.0


In [65]:
lifetime_df = lifetime_df[(lifetime_df["T"] > 0)]
lifetime_df = lifetime_df[lifetime_df["recency"] > 0]

In [66]:
lifetime_df.describe()

,frequency,recency,T
count,138584.000000,138584.000000,138584.000000
mean,47.350069,751.047596,793.607797
std,515.542278,671.411639,664.529458
min,2.000000,1.000000,1.000000
25%,5.000000,169.000000,220.000000
50%,12.000000,584.000000,622.000000
75%,34.000000,1176.000000,1213.000000
max,146454.000000,3044.000000,3045.000000


In [ ]:
# similar API to scikit-learn and lifelines.
bgf = BetaGeoFitter(penalizer_coef=0.0)
bgf.fit(lifetime_df['frequency'], lifetime_df['recency'], lifetime_df['T'])
print(bgf)

In [ ]:
plot_frequency_recency_matrix(bgf)

In [ ]:
from lifetimes.plotting import plot_probability_alive_matrix

plot_probability_alive_matrix(bgf)